<a href="https://colab.research.google.com/github/vaibhavsaini5911/Cattle-Overrun-Prevention-System-COPS-/blob/master/Train_YoloV3_Multiple.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Connect google drive**

In [1]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

Thu Sep 24 20:17:37 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.66       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   64C    P8    11W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Mounted at /content/gdrive
'663fbc4d-aef9-4e57-9ef9-4bb153b22090 (1).jpg'
 663fbc4d-aef9-4e57-9ef9-4bb153b22090.jpg
 6.gdoc
 6.odt
 7.gdoc
 7.odt
'Alkyl halides.pdf'
 b4cf67f9-6e13-43c5-b176-2e484d51d344.jpg
 caninformedvotersenforcebettergovernance.pdf
 Ch06.pdf
 Class12_Chemistry_May_GeneralPrinciples_Assignment.pdf
 Class_12_Chemistry_Solutions_May_Assignment.pdf
 Classroom
'Colab Notebooks'
'DSLAB4 (1).docx'
 DSLAB4.docx
 DSLAB6.docx
 DSLABSS.docx
 Electricity-Magnetism.pdf
 Hostel
'Hostel holi'
 image1.jpeg
'IMG_0512 (1).JPG'
 IMG_0512.JPG
'IMG_0515 (1).JPG'
 IMG_0515.JPG
'IMG_0519 (1).JPG'
 IMG_0519.JPG
 IMG_0994.jpg
 IMG_0998.jpg
 IMG_1470.JPG
 IMG_1471.JPG
 IMG_3633.JPG
'IMG_3634 (1).JPG'
 IMG_3634.JPG
 IMG_3640.JPG
 IMG_3694.JPG
 IMG_3708.JPG
 IMG_3759.JPG
 IMG_3761.JPG
 IMG_3822.JPG
 IMG_4159.JPG
 IMG_4166.JPG
 IMG_4169.JPG
 IMG_4172.JPG
 IMG_4185.JPG
 IMG_4200.JPG
 IMG_4215.JPG
'IMG_4219 (1).JPG'
 IMG_4219.JPG
'IMG_4233 (1).JPG'
 IMG_4233.JPG
'IMG_4234 (1).JPG'
 IMG_4234.JPG

**1) Clone, configure & compile Darknet**



In [3]:
# Clone
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 14309, done.
remote: Total 14309 (delta 0), reused 0 (delta 0), pack-reused 14309
Receiving objects: 100% (14309/14309), 12.86 MiB | 23.64 MiB/s, done.
Resolving deltas: 100% (9766/9766), done.


In [4]:
# Configure
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile

/content/darknet


In [5]:
# Compile
!make

mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:926:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1127:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
             ^~
./src/image_

**2) Configure yolov3.cfg file**

In [7]:
# Make a copy of yolov3.cfg
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [8]:
# Change lines in yolov3.cfg file
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 2000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=18@' cfg/yolov3_training.cfg

**3) Create .names and .data files**

In [9]:
!echo -e 'Elephant' > data/obj.names
!echo -e 'classes= 1\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/yolov3' > data/obj.data

**4) Save yolov3_training.cfg and obj.names files in Google drive**

In [10]:
!cp cfg/yolov3_training.cfg /mydrive/yolov3/yolov3_testing.cfg
!cp data/obj.names /mydrive/yolov3/classes.txt

**5) Create a folder and unzip image dataset**

In [13]:
#!mkdir data/obj
!unzip /mydrive/yolov3/images.zip -d data/obj

Archive:  /mydrive/yolov3/images.zip
  inflating: data/obj/00072e3556ad7e81.jpg  
 extracting: data/obj/00072e3556ad7e81.txt  
  inflating: data/obj/000bf3863580a0a0.jpg  
 extracting: data/obj/000bf3863580a0a0.txt  
  inflating: data/obj/000f0d2fe4e5fde4.jpg  
 extracting: data/obj/000f0d2fe4e5fde4.txt  
  inflating: data/obj/001b4f6d9aedff6f.jpg  
  inflating: data/obj/001b4f6d9aedff6f.txt  
 extracting: data/obj/0032f42e15e82f90.jpg  
  inflating: data/obj/0032f42e15e82f90.txt  
  inflating: data/obj/00334122f228bb4d.jpg  
 extracting: data/obj/00334122f228bb4d.txt  
  inflating: data/obj/00371870727e83d8.jpg  
  inflating: data/obj/00371870727e83d8.txt  
  inflating: data/obj/003870d361518c99.jpg  
 extracting: data/obj/003870d361518c99.txt  
  inflating: data/obj/003a5d2cbb797b7c.jpg  
 extracting: data/obj/003a5d2cbb797b7c.txt  
 extracting: data/obj/00408077fc97698e.jpg  
  inflating: data/obj/00408077fc97698e.txt  
  inflating: data/obj/0062f2f5243acf12.jpg  
 extracting: data/

**6) Create train.txt file**

In [15]:
import glob
images_list = glob.glob("data/obj/*.jpg")
with open("data/train.txt", "w") as f:
    f.write("\n".join(images_list))

**7) Download pre-trained weights for the convolutional layers file**

In [16]:
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2020-09-24 20:26:35--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M   313KB/s    in 11m 2s  

2020-09-24 20:37:38 (240 KB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



**8) Start training**

In [17]:
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show
# Uncomment below and comment above to re-start your training from last saved weights
#!./darknet detector train data/obj.data cfg/yolov3_training.cfg /mydrive/yolov3/yolov3_training_last.weights -dont_show

Streaming output truncated to the last 5000 lines.

 1926: 0.592305, 0.445982 avg loss, 0.001000 rate, 10.270546 seconds, 123264 images, 0.323456 hours left
Loaded: 0.000040 seconds
v3 (mse loss, Normalizer: (iou: 0.75, cls: 1.00) Region 82 Avg (IOU: 0.849749, GIOU: 0.846616), Class: 0.999286, Obj: 0.954675, No Obj: 0.001907, .5R: 1.000000, .75R: 1.000000, count: 4, class_loss = 0.003987, iou_loss = 0.167104, total_loss = 0.171092 
v3 (mse loss, Normalizer: (iou: 0.75, cls: 1.00) Region 94 Avg (IOU: 0.000000, GIOU: 0.000000), Class: 0.000000, Obj: 0.000000, No Obj: 0.000008, .5R: 0.000000, .75R: 0.000000, count: 1, class_loss = 0.000117, iou_loss = 0.000000, total_loss = 0.000117 
v3 (mse loss, Normalizer: (iou: 0.75, cls: 1.00) Region 106 Avg (IOU: 0.000000, GIOU: 0.000000), Class: 0.000000, Obj: 0.000000, No Obj: 0.000001, .5R: 0.000000, .75R: 0.000000, count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.000000 
 total_bbox = 223943, rewritten_bbox = 0.359020 % 
v3 (